In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))
from os import environ
environ['PYGAME_HIDE_SUPPORT_PROMPT'] = '1'
import pygame  # it is important to import pygame after that
import time

#Pandas Settings
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
def scanner():
    
    url = 'http://18.231.88.179/sites/scanner.php?timeI=0&timeF=0'      
    link = requests.get(url)
    page = BeautifulSoup(link.text, 'html.parser')

    jogo = page.find_all('tr')
    head = page.find_all('',{'class':'text-center'})
    tam_head = len(page.find_all('',{'class':'text-center'}))

    # para dá toque de alerta quando chamada a função de toque
    pygame.init()
    pygame.mixer.music.load('toque.mpeg')
    
    # flag para indicar quando a função deve parar
    func = True
    if (len(jogo) - 1) == 0:
        func = False
        
    # varredura de jogos
    for i in range(1,len(jogo)):
        print(f'jogo {i}\n')

        # flag para garantir a nao repetição de dados em um mesmo 
        # período de tempo do mesmo jogo na hora de salvá-los
        
        
        # varredura de atributos do jogo
        for j in range(11):

            # caminho de cada dado mudando apenas o "j" para cada dado
            dado = jogo[i].find_all('td')[j]

            # tempo de jogo
            if j == 0:
                if (dado.text == 'HT') or (dado.text == 'FT'):
                    tempo = dado.text
                else:
                    tempo = int(dado.text.split("'")[0].split('+')[0])
                print(f'tempo {tempo}\n')

            # placar
            elif j == 1:
                placar = dado.text.replace(' - ','x')
                print(f'placar {placar}\n')

            # equipes
            elif j == 2:
                times = dado.text
                print(f'equipes: {times}\n')

            # chutes no gol
            elif j == 3: 
                chutesnogolcasa = dado.find('div').text

                # separando atributos de casa e visitante
                if (dado.text.split(chutesnogolcasa)[1]) == '':
                    chutesnogolvisitante = chutesnogolcasa
                else:
                    chutesnogolvisitante = dado.text.split(chutesnogolcasa)[1]
                print(f'chutes no gol casa {chutesnogolcasa}')            
                print(f'chutes no gol visitante {chutesnogolvisitante}\n')

            # chutes fora do gol 
            elif j == 4:
                chutesforagolcasa = dado.find('div').text

                # separando atributos de casa e visitante
                if (dado.text.split(chutesforagolcasa)[1]) == '':
                    chutesforagolvisitante = chutesforagolcasa
                else:
                    chutesforagolvisitante = dado.text.split(chutesforagolcasa)[1]
                print(f'chutes fora do gol casa {chutesforagolcasa}')            
                print(f'chutes fora do visitante {chutesforagolvisitante}\n')

            # escanteios
            elif j == 5:
                escanteioscasa = dado.find('div').text

                # separando atributos de casa e visitante
                if (dado.text.split(escanteioscasa)[1]) == '':
                    escanteiosvisitante = escanteioscasa
                else:
                    escanteiosvisitante = dado.text.split(escanteioscasa)[1]
                print(f'escanteios casa {escanteioscasa}')            
                print(f'escanteios visitante {escanteiosvisitante}\n')

            # ataques perigosos
            elif j == 6:
                ataquesperigososcasa = dado.find('div').text

                # separando atributos de casa e visitante
                if (dado.text.split(ataquesperigososcasa)[1]) == '':
                    ataquesperigososvisitante = ataquesperigososcasa
                else:
                    ataquesperigososvisitante = dado.text.split(ataquesperigososcasa)[1]
                print(f'ataques perigosos casa {ataquesperigososcasa}')            
                print(f'ataques perigosos visitante {ataquesperigososvisitante}\n')

            # posse de bola
            elif j == 7:
                possebolacasa = dado.find('div').text

                # separando atributos de casa e visitante
                if (dado.text.split(possebolacasa)[1]) == '':
                    possebolavisitante = possebolacasa
                else:
                    possebolavisitante = dado.text.split(possebolacasa)[1]
                print(f'posse de bola casa {possebolacasa}')            
                print(f'posse de bola visitante {possebolavisitante}\n')

            # PI 2
            elif j == 8:
                pi2casa = dado.find('div').text

                # separando atributos de casa e visitante
                if (dado.text.split(pi2casa)[1]) == '':
                    pi2visitante = pi2casa
                else:
                    pi2visitante = dado.text.split(pi2casa)[1]
                print(f'PI2 casa {pi2casa}')            
                print(f'PI2 visitante {pi2visitante}\n')

            # PI 1
            elif j == 9:
                pi1casa = dado.find('div').text

                # separando atributos de casa e visitante
                if (dado.text.split(pi1casa)[1]) == '':
                    pi1visitante = pi1casa
                else:
                    pi1visitante = dado.text.split(pi1casa)[1]
                print(f'PI1 casa {pi1casa}')            
                print(f'PI1 visitante {pi1visitante}\n')

            # link com odds
            elif j == 10:
                linkodds = 'http://18.231.88.179/sites/' + dado.find('a').get('href').split('/')[1]
                print('link com as odds')
                print(linkodds)

                # pegando as odds e informações do campeonato se tiver
                url = linkodds
                link = requests.get(url)
                page2 = BeautifulSoup(link.text, 'html.parser')
                data = page2.find('h6').text.split('Atualizado em ')[1][0:10]
                print(data)

        
        # CRIANDO UM DATAFRAME COM OS DADOS DA PARTIDA A CADA 10 MIN E ARMAZENANDO PARA FORMAR UMA BASE DE DADOS
        if (tempo == 'HT' or tempo == 'FT' or tempo % 10 == 0 or tempo == 88 or tempo == 89 or tempo == 82) and (tempo != 0) :
        
            # ler o dataframe anterior para depois concatenar com o novo
            try:
                anterior_df = pd.read_csv('dados_soccer_stats.csv')
            except:
                anterior_df = pd.DataFrame()
                pass

            # criando dataframe novo a cada 10 min
            atual_df = pd.DataFrame(
                {'tempo':[tempo], 'times':[times]}
            ) 
            
            
            '''{'tempo':[tempo], 'placar':[placar], 'times':[times], 'chutesnogolgolcasa':[chutesnogolcasa],
                 'chutesnogolvisitante':[chutesnogolvisitante], 'chutesforagolcasa':[chutesforagolcasa],
                 'chutesforagolvisitante':[chutesforagolvisitante], 'escanteioscasa':[escanteioscasa],
                 'escanteiosvisitante':[escanteiosvisitante], 'ataquesperigososcasa':[ataquesperigososcasa],
                 'ataquesperigososvisitante':[ataquesperigososvisitante], 'possecasa':[possecasa], 'possevisitante':[possevisitante],
                 'pi2casa':[pi2casa], 'pi2visitante':[pi2visitante], 'pi1casa':[pi1casa], 'pi1visitante':[pi1visitante]}
            )''' 
            pass
        
            # colocando o dataframe anterior e o atual em uma lista
            list_frames = [anterior_df, atual_df]
    
            # concatenando a lista de dataframe em um único dataframe
            final_df = pd.concat(list_frames, sort=False)
            
            try:
                # eliminando coluna indesejada                
                final_df.drop(['Unnamed: 0'], inplace=True, axis=1)

                # filtrando o tempo em questão para analisar se os atributos referentes aos times em questão já foram armazenados
                filtro_df = final_df[final_df['tempo'] == tempo]
                filtro_df = filtro_df[filtro_df['times'] == times]
        
                # se o de tempo ta vazio, o de times nunca vai ter aquele time de um filtro vazio e portanto o filtro continua vazio
                # se o de tempo nao ta vazio, tem que olhar se tem aqueles times em questão naquele tempo
                # so entra no if se ou filtro do tempo tiver vazio, ou o de times ou os dois
                if filtro_df.empty:
               
                    # atualizando o indice
                    final_df = final_df.reset_index(drop=True)

                    # salvando o dataframe construído a cada 10 min de cada partida
                    final_df.to_csv('dados_soccer_stats.csv')
                    print('salvei a partir do try')

                    
            except:
                # atualizando o indice
                final_df = final_df.reset_index(drop=True)
                
                # salvando o dataframe construído a cada 10 min de cada partida
                final_df.to_csv('dados_soccer_stats.csv')
                print('salvei a partir do except')
                pass
            
            
            
        # aviso sonoro 

        # ambas marcam
        if tempo != 'HT' and tempo != 'FT':
            if (int(chutesnogolcasa) >= 3) and (placar == '0x1' or placar == '0x2' or placar == '0x3') and (int(tempo <= 45)):
                print('GRANDE CHANCE PARA AMBAS')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)

            elif (int(chutesnogolvisitante) >= 3) and (placar == '1x0' or placar == '2x0' or placar == '3x0') and (int(tempo <= 45)):
                print('GRANDE CHANCE PARA AMBAS')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)

        else:
            if (int(chutesnogolcasa) >= 3) and (placar == '0x1' or placar == '0x2' or placar == '0x3') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA AMBAS')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)

            elif (int(chutesnogolvisitante) >= 3) and (placar == '1x0' or placar == '2x0' or placar == '3x0') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA AMBAS')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)



        # over gols
        if tempo != 'HT' and tempo != 'FT':
            if (int(chutesnogolcasa) >= 3) and (int(chutesnogolvisitante) >= 3) and (placar == '1x0' or placar == '2x0' or placar == '0x1' or placar == '0x2') and (int(tempo <= 45)):
                print('GRANDE CHANCE PARA OVER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)

            elif (int(chutesnogolcasa) >= 2) and (int(chutesnogolvisitante) >= 2) and (int(tempo <= 20)):
                print('GRANDE CHANCE PARA OVER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)

        else:
            if (int(chutesnogolcasa) >= 3) and (int(chutesnogolvisitante) >= 3) and (placar == '1x0' or placar == '2x0' or placar == '0x1' or placar == '0x2') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA OVER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)




        # under gols
        if tempo != 'HT' and tempo != 'FT':
            if (int(chutesnogolcasa) >= 2) and (int(chutesnogolvisitante) == 0) and (int(tempo) >= 35 and int(tempo) <= 45) and (placar == '1x0' or placar == '2x0'):
                print('GRANDE CHANCE PARA UNDER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)

            elif (int(chutesnogolvisitante) >= 2) and (int(chutesnogolcasa) == 0) and (int(tempo >= 35) and int(tempo <= 45)) and (placar == '0x1' or placar == '0x2'):
                print('GRANDE CHANCE PARA UNDER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)

        else:
            if (int(chutesnogolcasa) >= 2) and (int(chutesnogolvisitante) == 0) and (placar == '1x0' or placar == '2x0') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA UNDER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)

            elif (int(chutesnogolvisitante) >= 2) and (int(chutesnogolcasa) == 0) and (placar == '0x1' or placar == '0x2') and (tempo == 'HT'):
                print('GRANDE CHANCE PARA UNDER')
                print(times)

                # toca um áudio
                pygame.mixer.music.play()
                time.sleep(3)


        print('\n\n')
    return func

In [6]:
# chamando a função para fazer scanner in live
func = True
count = 0
while (func == True or count < 3):
    try:
        func = scanner()
        if func == False:
            count += 1
            time.sleep(1)
        else:
            count = 0
            time.sleep(15)
    except:
        time.sleep(30)
        continue
print('Nenhum jogo ao vivo neste momento.')

Nenhum jogo ao vivo neste momento.
